In [1]:
import os
import sys
import numpy as np
from skimage import io
import itk
from scipy import ndimage
from itkwidgets import compare, checkerboard, view

In [2]:
DATA = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/CH1'
REGDATA = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'

In [6]:
filename = 'allen_50um_sagittal.tif'
fixedFilepath = os.path.join(REGDATA, filename)
fixedImage = itk.imread(fixedFilepath, itk.F)

In [7]:
movingFilepath = os.path.join(DATA, 'aligned_volume.256.tif')
movingImage = itk.imread(movingFilepath, itk.F)

In [9]:
um = 50
# init transform start
# Translate to roughly position sample data on top of CCF data
init_transform = itk.VersorRigid3DTransform[itk.D].New()  # Represents 3D rigid transformation with unit quaternion
init_transform.SetIdentity()
transform_initializer = itk.CenteredVersorTransformInitializer[
    type(fixedImage), type(movingImage)
].New()
transform_initializer.SetFixedImage(fixedImage)
transform_initializer.SetMovingImage(movingImage)
transform_initializer.SetTransform(init_transform)
transform_initializer.GeometryOn()  # We compute translation between the center of each image
transform_initializer.ComputeRotationOff()  # We have previously verified that spatial orientation aligns
transform_initializer.InitializeTransform()
# initializer maps from the fixed image to the moving image,
# whereas we want to map from the moving image to the fixed image.
init_transform = init_transform.GetInverseTransform()
# init transform end
# Apply translation without resampling the image by updating the image origin directly
change_information_filter = itk.ChangeInformationImageFilter[type(movingImage)].New()
change_information_filter.SetInput(movingImage)
change_information_filter.SetOutputOrigin(
    init_transform.TransformPoint(itk.origin(movingImage))
)
change_information_filter.ChangeOriginOn()
change_information_filter.UpdateOutputInformation()
source_image_init = change_information_filter.GetOutput()
# end apply translation        

parameter_object = itk.ParameterObject.New()
rigid_parameter_map = parameter_object.GetDefaultParameterMap('rigid')
affine_parameter_map = parameter_object.GetDefaultParameterMap('affine')
bspline_parameter_map = parameter_object.GetDefaultParameterMap("bspline")
bspline_parameter_map["FinalGridSpacingInVoxels"] = (f"{um}",)
parameter_object.AddParameterMap(rigid_parameter_map)
parameter_object.AddParameterMap(affine_parameter_map)
parameter_object.AddParameterMap(bspline_parameter_map)
parameter_object.RemoveParameter("FinalGridSpacingInPhysicalUnits")
registration_method = itk.ElastixRegistrationMethod[type(fixedImage), type(movingImage)
].New(
    fixed_image=fixedImage,
    moving_image=source_image_init,
    parameter_object=parameter_object,
    log_to_console=False,
)
registration_method.Update()
resultImage = registration_method.GetOutput()
registered_output = os.path.join(DATA, 'registered')
## write transformation 
os.makedirs(registered_output, exist_ok=True)
outputpath = os.path.join(registered_output, 'init-transform.hdf5')
itk.transformwrite([init_transform], outputpath)
for index in range(parameter_object.GetNumberOfParameterMaps()):
    outputpath = os.path.join(registered_output, f'elastix-transform.{index}.txt')
    registration_method.GetTransformParameterObject().WriteParameterFile(
        registration_method.GetTransformParameterObject().GetParameterMap(index), outputpath) 


In [10]:
type(resultImage)

itk.itkImagePython.itkImageF3

In [11]:
# 1- blue is fixed (Allen), 2 red is moving (DKXX)
view(resultImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [ ]:
Dimension = 3
InputPixelType = itk.F
OutputPixelType = itk.UC
InputImageType = itk.Image[InputPixelType, Dimension]
OutputImageType = itk.Image[OutputPixelType, Dimension]
castImageFilter = itk.CastImageFilter[InputImageType, OutputImageType].New()
outpath = os.path.join(REGDATA, 'allen_test_affine_bpsline.tif')
#writer = itk.ImageFileWriter[OutputImageType].New()
#writer.SetFileName(outpath)
#writer.SetInput(castImageFilter.GetOutput())
#writer.Update()

itk.imwrite(resultImage, outpath)